In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy
from datetime import datetime
import statsmodels.formula.api as smf

from matplotlib import style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
from pylab import mpl
import platform
from pandas.tseries.offsets import MonthEnd  # 用于处理月末日期

# 绘图与字体设置
# 根据操作系统自动设置中文字体，确保绘图时中文能正常显示
system = platform.system()
if system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows使用黑体
    plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示为方块的问题
elif system == 'Darwin':  # macOS
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # macOS使用Arial Unicode
    plt.rcParams['axes.unicode_minus'] = False
else:  # Linux
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']
    plt.rcParams['axes.unicode_minus'] = False

# 设置输出为矢量图（SVG），使图表更清晰
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# 设置 jupyter 输出规则：显示所有列
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)

print(f"当前操作系统: {system}")
print(f"字体设置: {plt.rcParams['font.sans-serif']}")

当前操作系统: Windows
字体设置: ['SimHei']


In [ ]:
#读取个股交易数据 
cross = pd.read_csv('E:\BaiduNetdiskDownload/TRD_Mnth202509.csv')

# 格式化日期：将交易月转换为该月的最后一天，方便后续对齐
cross['month'] = pd.to_datetime(cross['Trdmnt'], format='%Y-%m') + MonthEnd(1)

# 格式化股票代码：补齐为6位字符串（如 '1' -> '000001'）
cross['Stkcd'] = cross['Stkcd'].apply(lambda x: '{:0>6}'.format(x))

# 重命名列以符合 Fama-French 的习惯命名
# Mretwd: 考虑现金红利的月个股回报率 -> Return
# Msmvosd: 月个股流通市值 -> floatingvalue
# Msmvttl: 月个股总市值 -> totalvalue
cross.rename(columns={'Mretwd': 'Return', 'Msmvosd': 'floatingvalue', 'Msmvttl': 'totalvalue'}, inplace=True)

# --- 2. 读取无风险利率数据 ---
rf_data = pd.read_csv('E:\BaiduNetdiskDownload/Marketret_mon_stock2024.csv')
rf_data['month'] = pd.to_datetime(rf_data['month'], format='%b %Y') + MonthEnd(1)
rf_data = rf_data[['month', 'rfmonth']] # 仅保留日期和无风险利率

#3. 数据合并与计算
# 将个股数据与无风险利率合并
cross = pd.merge(cross, rf_data, on='month', how='left')

# 计算上市时间：
# 按照股票和时间排序，计算这是该股票上市的第几个月
# 这一步通常用于剔除刚上市不久的股票（IPO效应）
cross = cross.sort_values(by=['Stkcd', 'month'])
cross['list_month'] = cross.groupby('Stkcd').cumcount() + 1

# 计算超额收益率 (Excess Return) = 个股收益 - 无风险利率
cross['ret'] = cross['Return'] - cross['rfmonth']

# 单位调整：市值数据通常单位较大，这里统一乘以1000（视原始数据单位而定）
cross['floatingvalue'] = cross['floatingvalue'] * 1000
cross['totalvalue'] = cross['totalvalue'] * 1000

cross.head()

In [ ]:
#生成下一期的收益率 (Next Month Return) 

# 目的：为了进行资产定价，我们需要用 当月(t) 的因子特征 去匹配 下个月(t+1) 的收益率。

#构建全量的 (股票 x 月份) 索引防止中间有月份缺失（如停牌）
all_months = pd.DataFrame(cross['month'].unique(), columns=['month'])
all_stocks = pd.DataFrame(cross['Stkcd'].unique(), columns=['Stkcd'])
full_index = all_stocks.merge(all_months, how='cross') # 笛卡尔积

# 将原始数据合并到全量索引中
cross_full = full_index.merge(cross, on=['Stkcd', 'month'], how='left')
cross_full = cross_full.sort_values(['Stkcd', 'month'])

#  使用 shift(-1) 获取下个月的收益率
# groupy 保证了是在同一只股票内部进行位移
cross_full['next_ret'] = cross_full.groupby('Stkcd')['ret'].shift(-1)

# 还原数据，只保留原始存在的行，但带上了 next_ret
cross = cross.merge(cross_full[['Stkcd', 'month', 'next_ret']], 
                    on=['Stkcd', 'month'], how='right')

#  计算过去12个月的累计交易天数
# 这通常用于剔除长期停牌或交易不活跃的股票
cross['Cumsum_tradingday'] = cross.groupby('Stkcd')['Ndaytrd'].transform(lambda x: x.rolling(window=12, min_periods=1).sum())

print("下月收益率构建完成。")

# 筛选时间范围：1995年之后的数据通常质量更好
cross = cross[(cross['month'] >= '1995-01-31') & (cross['month'] <= '2024-12-31')]
cross.head()

In [ ]:
#导入市场收益率 (MKT Factor)
Market_ret = pd.read_csv('E:\BaiduNetdiskDownload/Marketret_mon_stock2024.csv')
Market_ret['month'] = pd.to_datetime(Market_ret['month'], format='%b %Y') + MonthEnd(0) # 注意这里是MonthEnd(0)处理对其
Market_ret.set_index('month', inplace=True)
Market_ret.sort_index(inplace=True)
if 'Unnamed: 0' in Market_ret.columns:
    Market_ret = Market_ret.drop(columns=['Unnamed: 0'])
Market_ret.rename(columns={'ret': 'MKT'}, inplace=True)

# 将市场因子合并入主表
cross = pd.merge(cross, Market_ret[['MKT']], left_on='month', right_on='month', how='left')

# 导入价值因子数据 (EP Ratio: Earnings/Price)
# EP 是 PE (市盈率) 的倒数
EP = pd.read_csv('datasets/EP_individual_mon2024.csv')
EP['Stkcd'] = EP['Stkcd'].apply(lambda x: '{:0>6}'.format(x))

# 处理特殊的月份格式 (例如 1991.25 -> 1991年3月)
EP['year'] = EP['month'].astype(int)
EP['month_decimal'] = EP['month'] - EP['year']
# 逻辑：小数部分 * 12 四舍五入得到月份
EP['month_num'] = (EP['month_decimal'] * 12).round().astype(int) + 1

# 边界处理：如果四舍五入导致月份>12，年份+1
EP.loc[EP['month_num'] > 12, 'year'] += 1
EP.loc[EP['month_num'] > 12, 'month_num'] -= 12

EP['month'] = pd.to_datetime(EP['year'].astype(str) + '-' + EP['month_num'].astype(str) + '-01') + MonthEnd(1)
EP = EP[['Stkcd', 'month', 'ep', 'ep_recent']]

# 将 EP 数据合并入主表
cross = pd.merge(cross, EP, on=['Stkcd', 'month'], how='left')

In [ ]:
# 计算规模过滤阈值 (Size Filter)
# Fama-French 方法通常会去除市值最小的 30% 股票，以防止微小盘股的极端收益扭曲结果（壳价值污染）
fenweishu = pd.DataFrame(
    cross.groupby(['month'])['totalvalue'].quantile(0.3))
fenweishu.columns = ['fenweishu_guimo']

# 合并阈值并进行过滤
cross_new = pd.merge(cross, fenweishu, on='month', how='left')
# 只保留市值大于当月 30% 分位数的股票
cross_new = cross_new[cross_new['totalvalue'] > cross_new['fenweishu_guimo']]

#  进一步的数据清洗
# 剔除交易天数过少、刚上市(6个月内)、过去一年交易不活跃的股票
# 保留主板和创业板 (Markettype: 1=上海A, 4=深圳A, 16=创业板)
cross_new = cross_new[cross_new['Ndaytrd'] >= 12]
cross_new = cross_new[cross_new['list_month'] > 6]
cross_new = cross_new[cross_new['Cumsum_tradingday'] >= 120]
cross_new = cross_new[cross_new['Markettype'].isin([1, 4, 16])]
cross_new = cross_new.dropna(subset=['ep']) # 必须有估值数据

# 计算 2x3 分组的断点 (Breakpoints) 
# 规模因子 (Size): 使用中位数 (0.5) 将股票分为 大(B) 和 小(S)
guimo = cross_new.groupby(['month'])['totalvalue'].quantile(0.5).to_frame()
guimo.columns = ['guimo']

# 价值因子 (Value): 使用 30% 和 70% 分位数将股票分为 成长(G), 中性(M), 价值(V)
# 注意：EP 越高代表越是价值股（High Value），EP 越低代表越是成长股（Low Value/Growth）
jiazhi = cross_new.groupby(['month'])['ep'].quantile([0.3, 0.7]).to_frame()
jiazhi = jiazhi.unstack()
jiazhi.columns = ['jiazhi_30', 'jiazhi_70']

# 将断点合并回数据
cross_new = pd.merge(cross_new, guimo, on='month', how='left')
cross_new = pd.merge(cross_new, jiazhi, on='month', how='left')

# --- 4. 股票打标签 (Grouping) ---
# Size: B (Big), S (Small)
cross_new['size'] = np.where(cross_new['totalvalue'] > cross_new['guimo'], 'B', 'S')

# Value: V (Value, High EP), M (Middle), G (Growth, Low EP)
cross_new['value'] = np.where(cross_new['ep'] > cross_new['jiazhi_70'], 'V',
                              np.where(cross_new['ep'] > cross_new['jiazhi_30'], 'M', 'G'))

# 确保下一期收益和市值不为空，用于计算加权收益
cross_new = cross_new.dropna(subset=['next_ret', 'totalvalue'])

In [ ]:
# 定义计算市值加权平均收益率的函数
def weighted_ret(group):
    # 使用 totalvalue 作为权重计算 next_ret 的加权平均
    return np.average(group['next_ret'], weights=group['totalvalue'])

def calc_portfolio_returns(data):
    """
    计算 2x3=6 个基础投资组合的月度加权收益率
    """
    portfolios = {}
    for size in ['S', 'B']:
        for value in ['V', 'M', 'G']:
            # 筛选属于该组的股票
            mask = (data['size'] == size) & (data['value'] == value)
            port_name = f'{size}{value}' # 例如: SV (Small Value)
            
            # 按月分组计算加权收益
            portfolios[port_name] = (data[mask]
                                   .groupby('month')
                                   .apply(weighted_ret, include_groups=False)
                                   .to_frame(name=port_name))
    return pd.concat(portfolios.values(), axis=1)
    
six_portfolio = calc_portfolio_returns(cross_new)

# 调整索引：因为 next_ret 是下一期的收益，所以这里的 return 实际上归属于 month + 1
six_portfolio.index = six_portfolio.index + MonthEnd(1)
six_portfolio = six_portfolio['2000-01':] # 截取 2000 年之后的数据进行分析

six_portfolio.head()

In [ ]:
def calc_factors(portfolios):
    """
    根据六个基础组合计算 SMB 和 HML 因子
    """
    # SMB (Small Minus Big): 市值因子
    # 逻辑: (小盘股三种组合的平均) - (大盘股三种组合的平均)
    # SMB = 1/3 (Small Value + Small Neutral + Small Growth) - 1/3 (Big Value + Big Neutral + Big Growth)
    smb = ((portfolios['SV'] + portfolios['SM'] + portfolios['SG'])/3 - 
           (portfolios['BV'] + portfolios['BM'] + portfolios['BG'])/3)
    
    # HML (High Minus Low): 价值因子
    # 逻辑: (价值股两种组合的平均) - (成长股两种组合的平均)
    # HML = 1/2 (Small Value + Big Value) - 1/2 (Small Growth + Big Growth)
    hml = ((portfolios['SV'] + portfolios['BV'])/2 - 
           (portfolios['SG'] + portfolios['BG'])/2)
    
    return pd.DataFrame({
        'SMB': smb,
        'HML': hml
    })

factors = calc_factors(six_portfolio)

# 合并市场因子 MKT
factors = pd.merge(factors, Market_ret[['MKT']], left_index=True, right_index=True, how='left')
factors = factors[['MKT', 'SMB', 'HML']]

# 保存因子数据
factors.to_csv('datasets/factors_3f.csv')
factors.head()

In [ ]:
#构造 TotalValue (规模) 十分组投资策略
def calc_decile_portfolios(data):
    """
    根据总市值(totalvalue)构造十分组投资组合，测试规模效应
    P1 为市值最小组，P10 为市值最大组
    """
    portfolios = {}
    
    # 内部函数：为每个月的每只股票分配 0-9 的标签
    def assign_decile(group):
        # 计算 0.1 到 1.0 的分位数
        quantiles = [group['totalvalue'].quantile(i/10) for i in range(11)]
        # pd.cut 将数据分箱
        labels = pd.cut(group['totalvalue'], bins=quantiles, labels=False, include_lowest=True, duplicates='drop')
        return labels
    
    data_copy = data.copy()
    # 按月分组进行打标
    data_copy['decile'] = data_copy.groupby('month', group_keys=False).apply(
        lambda x: assign_decile(x), include_groups=False
    )
    
    # 计算每个十分组的市值加权收益率
    for i in range(10):
        decile_name = f'P{i+1}' # P1, P2 ... P10
        portfolios[decile_name] = (
            data_copy[data_copy['decile'] == i]
            .groupby('month')
            .apply(lambda x: np.average(x['next_ret'], weights=x['totalvalue']), include_groups=False)
            .to_frame(name=decile_name)
        )
    
    return pd.concat(portfolios.values(), axis=1)

# 计算十分组收益
decile_portfolios = calc_decile_portfolios(cross_new)
decile_portfolios.index = decile_portfolios.index + MonthEnd(1)
decile_portfolios = decile_portfolios['2000-01':]

print("规模因子十分组投资组合描述统计:")
decile_portfolios.describe()

# --- 回归分析：验证规模效应 ---
# 构建多空组合：做多小市值(P1)，做空大市值(P10)
decile_portfolios['size_portfolio'] = decile_portfolios['P1'] - decile_portfolios['P10']

# 合并 FF3 因子进行调整
decile_portfolios = pd.merge(decile_portfolios, factors, left_index=True, right_index=True, how='left')

# 1. 单截距回归 (检验未经风险调整的超额收益)
# 使用 HAC (Newey-West) 标准误以修正自相关和异方差
model_size = smf.ols('size_portfolio ~ 1',
                 data=decile_portfolios['2000-01':'2024-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print("--- 规模效应多空组合 (P1-P10) 均值检验 ---")
print(model_size.summary())

# 2. FF3 因子回归 (检验风险调整后的 Alpha)
model_size_3factors = smf.ols('size_portfolio ~ MKT + SMB + HML',
                 data=decile_portfolios['2000-01':'2024-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print("--- 规模效应多空组合 FF3 调整后 Alpha ---")
print(model_size_3factors.summary())